# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [5]:
!pip install transformers --quiet
!pip install datasets --quiet
!pip install torch --quiet
!pip install torch torchvision torchaudio --quiet
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
import torch
print(torch.__version__)

2.0.1


In [2]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
    )
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [3]:
for param in model.base_model.parameters():
    param.requires_grad = False

In [4]:
#common parts of both models (standard and finetuned)

splits = ['train', 'test']
ds = load_dataset("imdb", split=splits)

#ds = {split: ds for split in splits in zip(ds, splits)}
# Convert list to dictionary
ds = {split: ds[i] for i, split in enumerate(splits)}

for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(500))

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation = True, padding=True)

tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)
    
# Compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

NameError: name 'load_dataset' is not defined

In [5]:
### with normal pytorch

In [6]:
'''
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import load_dataset

# Initialize model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,  # Binary classification (positive/negative)
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Freeze base model parameters to prevent them from being updated during training
for param in model.base_model.parameters():
    param.requires_grad = False

# Load and prepare dataset
splits = ['train', 'test']
ds = {split: load_dataset("imdb", split=split) for split in splits}

# Thin out the dataset for faster execution (only use first 500 examples from each split)
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(500))

# Tokenization function to apply truncation and padding to the dataset
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

# Tokenize dataset using the preprocess function
tokenized_ds = {split: ds[split].map(preprocess_function, batched=True) for split in splits}

# Convert datasets to PyTorch dataloaders using DataCollatorWithPadding for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer)

train_loader = DataLoader(tokenized_ds['train'], batch_size=4, collate_fn=data_collator)
test_loader = DataLoader(tokenized_ds['test'], batch_size=4, collate_fn=data_collator)

# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=2e-3)

# Training loop
epochs = 1
model.train()  # Set the model to training mode

for epoch in range(epochs):
    total_loss = 0
    for batch in train_loader:
        # Move inputs and labels to the device used by the model (GPU or CPU)
        inputs = {k: v.to(model.device) for k, v in batch.items() if k in tokenizer.model_input_names}
        labels = batch['labels'].to(model.device)
        
        # Zero the gradients before the forward pass
        optimizer.zero_grad()
        
        # Perform the forward pass
        outputs = model(**inputs)
        
        # Compute the loss
        loss = loss_fn(outputs.logits, labels)
        
        # Backpropagate the gradients
        loss.backward()
        
        # Update the model parameters
        optimizer.step()
        
        total_loss += loss.item()  # Accumulate the loss for reporting
    
    print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}')

# Evaluation loop
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient computation for evaluation
    for batch in test_loader:
        # Move inputs and labels to the device used by the model (GPU or CPU)
        inputs = {k: v.to(model.device) for k, v in batch.items() if k in tokenizer.model_input_names}
        labels = batch['labels'].to(model.device)
        
        # Perform the forward pass
        outputs = model(**inputs)
        
        # Get the predicted class with the highest logit
        _, predicted = torch.max(outputs.logits, 1)
        
        total += labels.size(0)  # Accumulate the total number of examples
        correct += (predicted == labels).sum().item()  # Accumulate the number of correct predictions

# Compute and print the accuracy
accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')
'''

'\nimport torch\nfrom torch import nn, optim\nfrom torch.utils.data import DataLoader\nfrom transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding\nfrom datasets import load_dataset\n\n# Initialize model and tokenizer\nmodel = AutoModelForSequenceClassification.from_pretrained(\n    "distilbert-base-uncased",\n    num_labels=2,  # Binary classification (positive/negative)\n    id2label={0: "NEGATIVE", 1: "POSITIVE"},\n    label2id={"NEGATIVE": 0, "POSITIVE": 1},\n)\ntokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")\n\n# Freeze base model parameters to prevent them from being updated during training\nfor param in model.base_model.parameters():\n    param.requires_grad = False\n\n# Load and prepare dataset\nsplits = [\'train\', \'test\']\nds = {split: load_dataset("imdb", split=split) for split in splits}\n\n# Thin out the dataset for faster execution (only use first 500 examples from each split)\nfor split in splits:\n    ds[spl

In [7]:
#with hugginface trainer

In [8]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    # Training arguments
    args=TrainingArguments(
        output_dir="./sentiment_analysis",
        learning_rate=2e-3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()

# Save the fine-tuned model and tokenizer
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.484682,0.790000


Checkpoint destination directory ./sentiment_analysis/checkpoint-125 already exists and is non-empty.Saving will proceed but saved results may be invalid.


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [9]:
trainer.evaluate()

{'eval_loss': 0.48468244075775146,
 'eval_accuracy': 0.79,
 'eval_runtime': 8.0441,
 'eval_samples_per_second': 62.157,
 'eval_steps_per_second': 15.539,
 'epoch': 1.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [10]:
# i instantiate the model before installing peft because it was generating errors
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
    )

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
!pip install peft --quiet
from peft import get_peft_model, LoraConfig, PeftModelForSequenceClassification, TaskType, AutoPeftModelForSequenceClassification

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
# let's examine the model structure
'''
(5): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            ))
'''
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [13]:
# PEFT model configuration
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=4,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=[
        "q_lin", "k_lin", "v_lin"
    ]
)

peft_model_configured = PeftModelForSequenceClassification(model, peft_config)

In [14]:
# Initialize the Trainer for the PEFT model
trainer_peft = Trainer(
    model=peft_model_configured,
    args=TrainingArguments(
        output_dir='./sentiment_analysis',
        learning_rate=2e-3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

# Train the PEFT model
trainer_peft.train()

# Evaluate the PEFT model
trainer_peft.evaluate()

# Save the fine-tuned PEFT model weights
peft_model_configured.save_pretrained("./peft_fine_tuned_model")
tokenizer.save_pretrained("./peft_fine_tuned_model") #same tokenizer as before

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.435292,0.858000


Checkpoint destination directory ./sentiment_analysis/checkpoint-125 already exists and is non-empty.Saving will proceed but saved results may be invalid.


('./peft_fine_tuned_model/tokenizer_config.json',
 './peft_fine_tuned_model/special_tokens_map.json',
 './peft_fine_tuned_model/vocab.txt',
 './peft_fine_tuned_model/added_tokens.json',
 './peft_fine_tuned_model/tokenizer.json')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [16]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define some example texts for inference
example_texts = [
    "This movie was great, I really enjoyed it.",
    "The plot was confusing and the acting was terrible.",
    "I didn't like this film at all.",
]

# Tokenize the examples
tokenized_input = tokenizer(example_texts, padding=True, truncation=True, return_tensors="pt")

# Move tokenized inputs to the same device as the model
tokenized_input = {key: value.to(device) for key, value in tokenized_input.items()}

# Move the model to the appropriate device
peft_model_configured.to(device)

# Perform inference
with torch.no_grad():
    outputs = peft_model_configured(**tokenized_input)

# Get predictions
predictions = torch.argmax(outputs.logits, dim=-1)

# Map label ids to labels
label_map = {0: "NEGATIVE", 1: "POSITIVE"}
predicted_labels = [label_map[pred.item()] for pred in predictions]

# Print results
for text, label in zip(example_texts, predicted_labels):
    print(f"Text: {text}")
    print(f"Predicted Sentiment: {label}")
    print()

Text: This movie was great, I really enjoyed it.
Predicted Sentiment: POSITIVE

Text: The plot was confusing and the acting was terrible.
Predicted Sentiment: NEGATIVE

Text: I didn't like this film at all.
Predicted Sentiment: NEGATIVE



### Reload and evaluate model

In [10]:
from peft import AutoPeftModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Load the saved PEFT model using AutoPeftModelForSequenceClassification
peft_model = AutoPeftModelForSequenceClassification.from_pretrained("./peft_fine_tuned_model")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Reload the ds
splits = ['train', 'test']
ds = load_dataset("imdb", split=splits)

#ds = {split: ds for split in splits in zip(ds, splits)}
# Convert list to dictionary
ds = {split: ds[i] for i, split in enumerate(splits)}

for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(500))

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation = True, padding=True)

tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)
    
# Compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [12]:
# Re-evaluate the PEFT model
trainer = Trainer(
    model=peft_model,
    args=TrainingArguments(
        output_dir="./sentiment_analysis",
        per_device_eval_batch_size=4,
    ),
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

# Re-evaluate the model
evaluation_results = trainer.evaluate()
print("Re-evaluation results:", evaluation_results)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Re-evaluation results: {'eval_loss': 0.43529194593429565, 'eval_accuracy': 0.858, 'eval_runtime': 10.1058, 'eval_samples_per_second': 49.477, 'eval_steps_per_second': 12.369}
